# Isolated Sources

- creation date 2024-03-20
- last update 2024-03-20
  
``Quantum Graphs``: https://tigress-web.princeton.edu/~lkelvin/pipelines/ 

``CalibrateTask``: 
https://github.com/lsst/pipe_tasks/blob/main/python/lsst/pipe/tasks/calibrate.py#L392-L399

 

In [ ]:
from lsst.daf.butler import Butler

import astropy.units as u
import numpy as np 
import pandas as pd
pd.set_option("display.max_columns", None)
from astropy.time import Time

import scipy.stats

import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt

import lsst.afw.display.rgb as afwRgb
import lsst.afw.image as afwImage
import lsst.geom as geom
import pickle
from scipy import interpolate

In [ ]:
plt.rcParams["figure.figsize"] = (8,8)
plt.rcParams["axes.labelsize"] = 'large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'large'
plt.rcParams['ytick.labelsize']= 'large'
plt.rcParams['legend.fontsize']=  12
plt.rcParams['font.size'] = 10

In [ ]:
dm_version="repos_w_2024_10" 
machine_name = os.uname().nodename
print(machine_name)
if 'sdf' in machine_name:
    #machine_name_usdf = 'sdfrome001'
    #machine_name_notebook platform = 'dagoret-nb'
    print("Set environement for USDF")
    subdir = f"repos/{dm_version}/rubinsimphot/src"
    newpythonpath = os.path.join(os.getenv("HOME"),subdir)
    print(subdir,newpythonpath)
    sys.path.append(newpythonpath)
    newpythonpath = os.path.join(os.getenv("HOME"),"rubin-user/RubinLSSTPhotometricCorrTuto/notebooks/lib")
    sys.path.append(newpythonpath)   
elif "dagoret-nb" in machine_name:
    print("Set environement for Rubin Platform at  USDF")
    newpythonpath = os.path.join(os.getenv("HOME"),f"repos/{dm_version}/rubinsimphot/src")
    sys.path.append(newpythonpath)
    print(newpythonpath)
    newpythonpath = os.path.join(os.getenv("HOME"),"rubin-user/RubinLSSTPhotometricCorrTuto/notebooks/lib")
    sys.path.append(newpythonpath)
elif 'mac' in machine_name:
    print("Be sure to run this notebook in conda environnement named conda_py310")
else:
    print("Your current machine name is {machine_name}. Check your python environment")

In [ ]:
from rubinsimphot.phot_utils import Bandpass, Sed
from rubinsimphot.data.data_sets import  get_data_dir

#README.md        darksky.dat      filter_r.dat     hardware_g.dat   hardware_y.dat   lens3.dat        total_g.dat      total_y.dat
#README_SOURCE.md detector.dat     filter_u.dat     hardware_i.dat   hardware_z.dat   m1.dat           total_i.dat      total_z.dat
#atmos_10.dat     filter_g.dat     filter_y.dat     hardware_r.dat   lens1.dat        m2.dat           total_r.dat      version_info
#atmos_std.dat    filter_i.dat     filter_z.dat     hardware_u.dat   lens2.dat        m3.dat           total_u.dat
hardware_filenames = ["hardware_u.dat","hardware_g.dat","hardware_r.dat","hardware_i.dat","hardware_z.dat","hardware_y.dat"] 
filter_filenames = ["filter_u.dat","filter_g.dat","filter_r.dat","filter_i.dat","filter_z.dat","filter_y.dat" ]
total_filenames = ["total_u.dat","total_g.dat","total_r.dat","total_i.dat","total_z.dat","total_y.dat" ]
filter_tagnames = ["u","g","r","i","z","y"]
Filter_tagnames = ["U","G","R","I","Z","Y"]
filtercolor_tagnames = ["u-g","g-r","r-i","i-z","z-y"]
Filtercolor_tagnames = ["U-G","G-R","R-I","I-Y","Z-Y"]
filter_color = ["b","g","r","orange","grey","k"]
NFILT=len(filter_filenames)

WLMIN=300.
WLMAX=1100.
WLBIN=1.
NWLBIN=int((WLMAX-WLMIN)/WLBIN)
WL=np.linspace(WLMIN,WLMAX,NWLBIN)

In [ ]:
#FILTERWL: precalculated array containing center, boundaries and width of each filter.
#index 0 : minimum wavelength of filter border
#index 1 : minimum wavelength of filter border
#index 2 : center wavelength of filter
#index 3 : filter width


FILTERWL = np.array([[ 324.03003755,  402.12765957,  363.59690349,   78.09762203],
       [ 392.11514393,  561.32665832,  473.54069923,  169.21151439],
       [ 542.3028786 ,  700.50062578,  619.49926767,  158.19774718],
       [ 681.47684606,  827.65957447,  752.01084117,  146.18272841],
       [ 808.63579474,  932.79098874,  868.488419  ,  124.15519399],
       [ 914.76846058, 1044.93116395,  969.10570859,  130.16270338]])

FILTERWL_auxtel = np.array([[ 352.7 ,  395.9 ,  374.3 ,   43.2 ],
                     [ 387.6 ,  566.2 ,  476.9 ,  178.6 ],
                     [ 541.4 ,  715.5 ,  628.45,  174.1 ],
                     [ 673.3 ,  870.9 ,  772.1 ,  197.6 ],
                     [ 805.6 , 1090.7 ,  948.15,  285.1 ]])


F0 = 3631.0 # Jy 1, Jy = 10^{-23} erg.cm^{-2}.s^{-1}.Hz^{-1}
Jy_to_ergcmm2sm1hzm1 = 1e-23
DT = 30.0 # seconds
gel = 1.1
#hP = 6.62607015E-34 # J⋅Hz−1
hP = 6.626196E-27
A  = np.pi*642.3**2 # cm2  Reff=6.423 m
A_auxtel  = 9636.0 # cm2

#ZPT_cont =  2.5 \log_{10} \left(\frac{F_0 A \Delta T}{g_{el} h} \right)
ZPTconst = 2.5*np.log10(F0*Jy_to_ergcmm2sm1hzm1*A*DT/gel/hP)

In [ ]:
!ls /home/d/dagoret/repos/repos_w_2024_10/rubinsimphot/src/rubin_sim_data/throughputs/auxtel

In [ ]:
fdir = get_data_dir()
bandpass_inst = {}
path_rubin_sim_throughput = os.path.join(fdir, 'throughputs', 'baseline')
for index,filename in enumerate(hardware_filenames):
    fullfilename=os.path.join(path_rubin_sim_throughput,filename)
    arr= np.loadtxt(fullfilename)
    # interpolate  filter transmission
    ff = interpolate.interp1d(x=arr[:,0], y=arr[:,1],fill_value="extrapolate")
    fname = filter_tagnames[index]
    bandpass_inst[fname] = Bandpass(wavelen=WL,sb=ff(WL))

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    axs.plot(WL,bandpass_inst[f].sb,color=filter_color[index]) 
    axs.fill_between(WL,bandpass_inst[f].sb,color=filter_color[index],alpha=0.2) 
    axs.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
    
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Total Rubin-LSST filter throughput")

In [ ]:
saveDir = "./lc_output_auxtelfall2023_spectro"
doSaveFigs = False  # set to True if you actually want to save the figures in the above dir

In [ ]:
# Set some selections for reliable data
minNumMatches = 3
minSnCalibFlux = 50  # 100

In [ ]:
# INSERT YOUR auxTel/LATISS collection and tract/band HERE
butlerRoot = "/repo/embargo"
# collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20231207/w_2023_49/PREOPS-4648"
collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"

collectionStr = collection.replace("/", "_")
instrument = "LATISS"
skymapName = "latiss_v1"

tract = 3864

calibFluxStr = "apFlux_35_0_instFlux"


## Selected Visits and Selected Nights

In [ ]:
file_selected_visits = "../data/202402/SelectedVisits_fall2023_tract_3864.csv"
file_selected_nights = "AuxtelSpectroData/AtmParamPerNightObsDict_fall2023.pkl"

In [ ]:
df_myselectedvisits = pd.read_csv(file_selected_visits,index_col=0)
my_selectedvisits = list(df_myselectedvisits.index)

In [ ]:
#move the visitid as a column not to loose it during the merge 
df_myselectedvisits.reset_index(inplace=True) 

In [ ]:
df_myselectedvisits

In [ ]:
# Check the night selected
df_myselectedvisits['nightObs'].unique()

### Make the dataframe holding the atmospheric parameters

In [ ]:
df_selnight = pd.DataFrame(columns=('nightObs', 'PWV', 'VAOD','Grey',"Ozone"))
with open(file_selected_nights, 'rb') as fp:
    selectedNightsDict  = pickle.load(fp)
idx=0
for key,value in selectedNightsDict.items():
    pwv = value['median']['median_param_atm']['md_PWV']
    vaod = value['median']['median_param_atm']['md_VAOD']
    grey = value['median']['median_param_atm']['md_grey']
    ozone = value['median']['median_param_atm']['md_ozone']
    df_selnight.loc[idx] = [int(key),pwv,vaod,grey,ozone]
    idx+=1

In [ ]:
# Convert the nightObs as an integer
df_selnight = df_selnight.astype({'nightObs':'int'})

In [ ]:
df_night_to_atm = df_selnight.copy().set_index('nightObs')

In [ ]:
df_night_to_atm

In [ ]:
df_night_to_atm.loc[20231127]

In [ ]:
df_selnight['nightObs'].unique()

## Merge the two datasets

- kepp the intersection

In [ ]:
df_merge = df_myselectedvisits.merge(df_selnight, left_on='nightObs',right_on='nightObs',how="inner")

In [ ]:
df_merge['nightObs'].unique()

In [ ]:
df_myselectedvisits = df_merge.copy()

In [ ]:
df_myselectedvisits

In [ ]:
my_selectedvisits = list(df_myselectedvisits.visitId)

In [ ]:
df_atm = df_myselectedvisits[["visitId","PWV","VAOD","Grey","Ozone"]]

## Config

In [ ]:
# Initiate butler from variables set above
butler = Butler(butlerRoot, collections=collection, instrument=instrument, skymap=skymapName)
camera = butler.get("camera", instrument=instrument)
skymap = butler.get("skyMap")
print("camera_name = {}".format(camera.getName()))
print("collection = {}".format(collection))

In [ ]:
# Try to get the Schema
data_product = "isolated_star_sources"
datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=collection, where= "instrument='LATISS'")
for i, ref in enumerate(datasetRefs):
    print(i,ref)
    butler_data = butler.get(ref)
    break

if not isinstance(butler_data, pd.core.frame.DataFrame):
    print(butler_data.getSchema())

## isolated_star_sources

The main starting point is the table of isolated_star_sources which has been constructed in step 2a from a catalog
Note that isolated star sources are associated to an object (a static starobject)
Here the work is doneband by band.
It is a good idea to work band by band.

In [ ]:
# Load in isolated_star_sources and trim to band of interest and select the tract
isolatedStarSourcesFull = butler.get("isolated_star_sources", tract=tract)
#isolatedStarSourcesFull = isolatedStarSourcesFull[isolatedStarSourcesFull["band"] == band]

In [ ]:
isolatedStarSourcesFull.head()

In [ ]:
isolatedStarSourcesFull.columns

## Select the visits in the preselected list

I add here my filter to select the visits I want to focus on from file_selected_visits = "../data/202402/SelectedVisits_fall2023_tract_3864.csv"
It include all bands

In [ ]:
def SelectByVisit(row):
    if row["visit"] in my_selectedvisits:
        return True
    else:
        return False

In [ ]:
isolatedStarSourcesFull["flag"] = isolatedStarSourcesFull.apply(SelectByVisit,axis=1,raw=False)

In [ ]:
isolatedStarSourcesFull = isolatedStarSourcesFull[isolatedStarSourcesFull["flag"]]
isolatedStarSourcesFull.drop("flag",axis=1,inplace=True)

In [ ]:
isolatedStarSourcesFull

## Add atmospheric quantities to the isolatedStarSourcesFull

In [ ]:
isolatedStarSourcesFull = isolatedStarSourcesFull.merge(df_atm, left_on='visit',right_on='visitId',how="inner")
isolatedStarSourcesFull["nightObs"] = isolatedStarSourcesFull.apply(lambda x: x['visit']//100_000, axis=1)
isolatedStarSourcesFull

In [ ]:
# Just to have a look at what's in the catalog:
isolatedStarSourcesFull[isolatedStarSourcesFull.index == 0]

In [ ]:
print("calibFluxStr = {},minSnCalibFlux = {}, minNumMatches = {}".format(calibFluxStr,minSnCalibFlux,minNumMatches))

In [ ]:
def convertVisitToDatestr(visit):

    num = visit//100_000
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return datestr

In [ ]:
def convertVisitToMJD(visit):
    return Time(convertVisitToDatestr(visit)).mjd

# Statistic on objects

In [ ]:
source_perobj_perband_pervisit = isolatedStarSourcesFull.groupby(by=["obj_index","band","visit"]).count()["ra"]

In [ ]:
source_perobj_perband_pervisit

## All objects

In [ ]:
list_of_object_id = sorted(isolatedStarSourcesFull["obj_index"].unique())
Nobj = len(list_of_object_id)

## Select one object

In [ ]:
cut_sel_object = isolatedStarSourcesFull.obj_index == 0

In [ ]:
isolatedStarSources_selected = isolatedStarSourcesFull[cut_sel_object]

In [ ]:
isolatedStarSources_selected.groupby(by="band").count()["index"] 

### Number of visit in y

In [ ]:
NYMIN = 50. # minimum number of visit in Y band
NBANDMIN = 4 # nimimum number of bands

In [ ]:
obj_idx_selected = []

for idx in range(Nobj):
    cut_sel_object = isolatedStarSourcesFull.obj_index == idx
    isolatedStarSources_selected = isolatedStarSourcesFull[cut_sel_object]
    
    ser= isolatedStarSources_selected.groupby(by="band").count()["index"]
    nband = len(ser)
    if "y" in ser.index and nband>= NBANDMIN:
        NY = ser["y"]
        if NY>NYMIN:
            #print(f"------------------------ {idx} ------------------------------")
            #print(ser,len(ser))
            obj_idx_selected.append(idx)
    
    

In [ ]:
print(obj_idx_selected)

In [ ]:
idx_obj_sel = 3
cut_sel_object = isolatedStarSourcesFull.obj_index == idx_obj_sel 
isolatedStarSources_sel = isolatedStarSourcesFull[cut_sel_object] 

In [ ]:
isolatedStarSources_sel

In [ ]:
isolatedStarSources_sel_g = isolatedStarSources_sel[isolatedStarSources_sel["band"] == "g"]
isolatedStarSources_sel_r = isolatedStarSources_sel[isolatedStarSources_sel["band"] == "r"]
isolatedStarSources_sel_z = isolatedStarSources_sel[isolatedStarSources_sel["band"] == "z"]
isolatedStarSources_sel_y = isolatedStarSources_sel[isolatedStarSources_sel["band"] == "y"]

In [ ]:
fig,axes = plt.subplots(1,4,figsize=(10,3),sharey=True)
axs= axes.flatten()
for idx,ax in enumerate(axs):
    if idx==0:
        isolatedStarSources_sel_g["apFlux_35_0_instFlux"].hist(bins=40,facecolor="g",ax=ax, orientation="horizontal")
    elif idx==1:
        isolatedStarSources_sel_r["apFlux_35_0_instFlux"].hist(bins=40,facecolor="r",ax=ax, orientation="horizontal")
    elif idx==2:
        isolatedStarSources_sel_z["apFlux_35_0_instFlux"].hist(bins=40,facecolor="grey",ax=ax, orientation="horizontal")
    elif idx==3:
        isolatedStarSources_sel_y["apFlux_35_0_instFlux"].hist(bins=40,facecolor="k",ax=ax, orientation="horizontal")
plt.tight_layout(h_pad=0)        
#plt.tight_layout()

In [ ]:
for idx_obj_sel in obj_idx_selected:
    cut_sel_object = isolatedStarSourcesFull.obj_index == idx_obj_sel 
    isolatedStarSources_sel = isolatedStarSourcesFull[cut_sel_object] 

    isolatedStarSources_sel_g = isolatedStarSources_sel[isolatedStarSources_sel["band"] == "g"]
    isolatedStarSources_sel_r = isolatedStarSources_sel[isolatedStarSources_sel["band"] == "r"]
    isolatedStarSources_sel_z = isolatedStarSources_sel[isolatedStarSources_sel["band"] == "z"]
    isolatedStarSources_sel_y = isolatedStarSources_sel[isolatedStarSources_sel["band"] == "y"]

    fig,axes = plt.subplots(1,4,figsize=(10,3),sharey=True)
    axs= axes.flatten()
    for idx,ax in enumerate(axs):
        if idx==0:
            isolatedStarSources_sel_g["apFlux_35_0_instFlux"].hist(bins=40,facecolor="g",ax=ax, orientation="horizontal")
            ax.legend("g")
        elif idx==1:
            isolatedStarSources_sel_r["apFlux_35_0_instFlux"].hist(bins=40,facecolor="r",ax=ax, orientation="horizontal")
            ax.legend("r")
        elif idx==2:
            isolatedStarSources_sel_z["apFlux_35_0_instFlux"].hist(bins=40,facecolor="grey",ax=ax, orientation="horizontal")
            ax.legend("z")
        elif idx==3:
            isolatedStarSources_sel_y["apFlux_35_0_instFlux"].hist(bins=40,facecolor="k",ax=ax, orientation="horizontal")
            ax.legend("y")
    plt.tight_layout(h_pad=0)  
    title =f"object {idx_obj_sel}"
    filename = f"sed_color_obj_{idx_obj_sel}.png"
    plt.suptitle(title,y=1.000)
    plt.savefig(filename)
    plt.show()
    